In [1]:
import sys
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn import set_config; set_config(display='diagram')

#sys.path.append("/Users/antonis/code/Ant-mel/legendary_game_recs/")
sys.path.append("/home/kieran/code/Ant-mel/legendary_game_recs/")


from preprocessing.preprocess_1_cleaning import *
from preprocessing.preprocess_2_features import *
from preprocessing.pipeline_the_unification import *
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import make_pipeline
from sklearn.pipeline import make_union
from sklearn.compose import make_column_transformer
import datetime
from sklearn.preprocessing import MultiLabelBinarizer

# Function that creates a list of features
def get_list_of_features(the_json):
    feature_dic = {"id":None,
        "aggregated_rating": None,
        "aggregated_rating_count": None,
        "game_engines": None,
        "game_modes": None,
        "player_perspectives": None,
        "themes": None,
        'rating': None}

    for key in the_json.keys():
        if type(the_json[key]) == list:
            for i in range(len(the_json[key])):
                value = the_json[key][i]['name']
                feature_dic.update({f'{key}': value})

        else:
            value = the_json[key]
            feature_dic.update({f'{key}': value})

    return feature_dic


def get_category_descriptions(json_dict):
    category_list = []

    if type(json_dict) == float:
        category_list = None
    else:
        for i in range(len(json_dict)):
            try:
                descriptions = json_dict[i]['content_descriptions']
                for i in range(len(descriptions)):
                    rating_descript = descriptions[i]['category']
                    category_list.append(rating_descript)

            except:
                pass

    return category_list

def get_names_from_dict(dict_list):
    list_of_items = []
    try:
        for i in range(len(dict_list)):
            item = dict_list[i]['name']
            list_of_items.append(item)
    except:
        list_of_items = None

    return list_of_items


In [ ]:
json_data = pd.read_json('../raw_data/api_json_0to30k')
json_data


In [ ]:
working_json_df = json_data


In [ ]:
dict_list = json_data.iloc[20000]['themes']


In [ ]:
get_names_from_dict(dict_list)


In [ ]:
json_data['themes'].apply(get_names_from_dict)


In [ ]:
def prepare_json_df(original_json_df):

    original_json_df['themes'] = original_json_df['themes'].apply(get_names_from_dict)
    original_json_df['game_modes'] = original_json_df['game_modes'].apply(get_names_from_dict)
    original_json_df['player_perspectives'] = original_json_df['player_perspectives'].apply(get_names_from_dict)
    original_json_df['game_engines'] = original_json_df['game_engines'].apply(get_names_from_dict)
    original_json_df['age_ratings'] = original_json_df['age_ratings'].apply(get_category_descriptions)

    return original_json_df.drop('multiplayer_modes', axis=1)


In [ ]:
prepare_json_df(json_data)


In [3]:
old_data = pd.read_csv('../raw_data/all_game_data_v1_corrected2.csv')


In [4]:
pipelined_up = pipeline_genre_ohe_only(old_data)


/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column]=pd.to_datetime(df[column])
/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_2_features.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dupe['gen'] = None
/home/kieran/code/Ant-mel/legendary_game_recs/preprocessing/preprocess_1_cleaning.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Tr

In [5]:
pipelined_up


,index,plays,playing,backlogs,wishlist,avg_review,genres,description,total_reviews,total_lists,...,Simulator,Puzzle,Shooter,Strategy,Platform,Arcade,Sport,gen_1,gen_2,gen_3
0,69,7,0,2,2,2.2,[Visual Novel],We all have multiple sides. Pick your poison! ...,1,4,...,0,0,0,0,0,0,0,0,0,1
1,166,80,1,82,297,3.3,"[Brawler, Indie, Shooter]",Anger Foot is a lightning-fast hard-bass blast...,8,104,...,0,0,1,0,0,0,0,0,0,1
2,377,9,0,5,1,2.4,"[Adventure, RPG, Simulator]",Life is Feudal: MMO is an open-world survival ...,0,3,...,1,0,0,0,0,0,0,0,0,1
3,599,10,3,9,41,4.5,"[Adventure, Fighting, Platform]",Play the history. Go behind the scenes of Jord...,5,12,...,0,0,0,0,1,0,0,0,0,1
4,634,16,0,26,155,2.1,"[Adventure, Indie, Puzzle]","Become Mia, a paranormal detective searching f...",12,27,...,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36815,118902,4,0,1,4,3.8,[Shooter],"In the year 2991 AD, the Galactic Federal Gove...",0,2,...,0,0,1,0,0,0,0,1,0,0
36816,119027,6,0,0,1,2.7,"[Adventure, Indie]","Level Up! is a Game created by Titch007uk, and...",1,2,...,0,0,0,0,0,0,0,0,1,0
36817,119068,3,0,0,0,2.2,[],Sea Battle is a board game everybody loves sin...,0,0,...,0,0,0,0,0,0,0,0,0,1
36818,119219,18,0,0,0,2.0,"[Indie, Simulator, Strategy]","Control minds, insert subliminal messages into...",2,7,...,1,0,0,1,0,0,0,0,0,1


In [6]:
pipelined_up.columns


Index(['index', 'plays', 'playing', 'backlogs', 'wishlist', 'avg_review',
       'genres', 'description', 'total_reviews', 'total_lists', 'Adventure',
       'Indie', 'RPG', 'Simulator', 'Puzzle', 'Shooter', 'Strategy',
       'Platform', 'Arcade', 'Sport', 'gen_1', 'gen_2', 'gen_3'],
      dtype='object')